In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [5]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(9006, 1995)

# run

## initialize log

In [6]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [7]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 0 0 

## run

In [ ]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
    ppl_train = get_perplexity(train_docs, topic_root)
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series    
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
  Time Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0  669  0  0   1037   1348  1095  0.02  0.39  0.43  0.71  0.68
1  766  1  0   1031   1339  1089  0.02  0.42  0.44  0.78  0.77
2  773  2  0   1022   1329  1082  0.02  0.42  0.45  0.80  0.79
3  769  3  0   1012   1321  1072  0.02  0.41  0.45  0.78  0.78
4  773  4  0   1002   1313  1063  0.02  0.41  0.45  0.77  0.76
5  841  5  0    995   1307  1056  0.02  0.42  0.45  0.79  0.79
6  850  6  0    990   1303  1051  0.02  0.42  0.45  0.79  0.78

 0 9006 487492.0 use write one get article know like make say think
   0-1 7630 135727.0 use write get one article know like make say work
     0-1-1 7593 1650.0 god evil good marriage know make believe free bible effect
     0-1-2 37 31.0 prohibit unix demand frequently assault highly industry manage criminal scale
   0-2 1084 44761.0 team game play player win year get go hockey season
     0-2-1 1084 311.0 houston city tom los braves philadelphia francisco louis red daniel
   0-3 12 1401.0 appear copy art vs cover new annual issue rider app
     0-3-2 12 10.0 limited rom dc society black contest want red various legal
   0-4 248 19979.0 ground space wire use mission orbit water outlet nasa one
     0-4-1 247 463.0 room lunar program near space treatment medical new health month
     0-4-2 1 0.0 bhj series ram dave knowledge prevent james throw washington legal
   0-5 11 1473.0 drive disk rom dos controller hard feature bio support floppy
     0-5-1 11 0.0 bhj series ram dave knowledg